In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath("..")  
sys.path.append(PROJECT_ROOT)

# Para asegurar también src/
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
sys.path.append(SRC_PATH)

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from src.fer.data.datasets import AlbumentationsImageFolder  
from src.fer.models.cnn.model import EmotionCNN          

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
base_dir = os.path.join(PROJECT_ROOT, "data", "raw")
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")

batch_size = 32  
train_ds = AlbumentationsImageFolder(train_dir, split="train")
val_ds   = AlbumentationsImageFolder(val_dir, split="val")

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)

len(train_ds), len(val_ds)


In [ ]:
num_classes = 7

model = EmotionCNN(num_classes=num_classes, in_channels=1).to(device)

criterion = nn.CrossEntropyLoss()

# 🔹 LR igual, pero añadimos weight_decay para regularizar un poco
learning_rate = 1e-3
weight_decay = 1e-4

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)

# El T_max lo vamos a igualar al número de épocas (lo definimos después)
# De momento ponemos un valor "dummy", luego lo re-definimos cuando tengamos num_epochs
scheduler = None



EmotionCNN(
  (block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), strid

In [ ]:
from tqdm.auto import tqdm

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Train", leave=False)

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)             # [B, 7]
        loss = criterion(outputs, labels)   # CE

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # accuracy
        _, preds = torch.max(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix(
            loss=loss.item(),
            acc=correct / total if total > 0 else 0.0
        )

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [ ]:
@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [ ]:
num_epochs = 25

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

best_val_acc = 0.0
history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch {epoch}/{num_epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = evaluate(model, val_loader, criterion, device)

    scheduler.step()

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(
        f"Train  - loss: {train_loss:.4f}, acc: {train_acc:.4f}\n"
        f"Val    - loss: {val_loss:.4f}, acc: {val_acc:.4f}"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "cnn_best_exp1.pt")
        print(f"Nuevo mejor modelo guardado (val_acc = {best_val_acc:.4f})")



Epoch 1/10


Train:   0%|          | 0/898 [00:09<?, ?it/s]

Train  - loss: 1.8225, acc: 0.2406
Val    - loss: 1.7753, acc: 0.2647
 Nuevo mejor modelo guardado (val_acc = 0.2647)

Epoch 2/10


Train:   0%|          | 0/898 [00:10<?, ?it/s]

Train  - loss: 1.7874, acc: 0.2585
Val    - loss: 1.7523, acc: 0.2859
 Nuevo mejor modelo guardado (val_acc = 0.2859)

Epoch 3/10


...

Train:   0%|          | 0/898 [00:12<?, ?it/s]

Train  - loss: 1.6291, acc: 0.3410
Val    - loss: 1.5635, acc: 0.3831
 Nuevo mejor modelo guardado (val_acc = 0.3831)

Epoch 9/10


Train:   0%|          | 0/898 [00:14<?, ?it/s]

Train  - loss: 1.6107, acc: 0.3511
Val    - loss: 1.5561, acc: 0.3881
 Nuevo mejor modelo guardado (val_acc = 0.3881)

Epoch 10/10


Train:   0%|          | 0/898 [00:12<?, ?it/s]

In [ ]:
class_names = train_ds.classes  

model.eval()
images, labels = next(iter(val_loader))
images = images.to(device)
labels = labels.to(device)

with torch.no_grad():
    outputs = model(images)
    _, preds = torch.max(outputs, dim=1)

list(zip([class_names[l.item()] for l in labels[:10]],
         [class_names[p.item()] for p in preds[:10]]))
